In [16]:
from collections import defaultdict
import random
"""
Example-to-Feature conversion methods
Modified from
https://github.com/salesforce/cos-e/blob/master/code/generation/train_commonsenseqa_v1.0.py and ""_v1.11.py (identical)
as well as Tensorflow code for WTF?: 
https://github.com/google-research/google-research/blob/master/wt5/wt5/preprocessors.py
"""
# This code is based on https://github.com/allenai/label_rationale_association/blob/main/feature_conversion_methods.py

unified_qa_esnli_label_mapping = {0: 'yes', 1: 'maybe', 2: 'no'}
unified_qa_esnli_label_mapping_upper = {0: 'Yes', 1: 'Maybe', 2: 'No'} 
wt5_esnli_label_mapping = {0: 'entailment', 1: 'neutral', 2: 'contradiction'} 
unified_qa_sbic_label_mapping = {"offensive": 'Yes', "not offensive": 'No'}

def format_instance(
        example,
        tokenizer,
        explanation_sep,
        max_seq_length=None,
        datasource=None,
        io_format=None, 
):
    assert datasource in {"cos_e", "esnli", "sbic", "sensemaking", "ecqa"}

    if datasource in ["cos_e", "ecqa"]:
        input_string, answer_string = cqa_formatting(example, io_format, explanation_sep, datasource)
    elif datasource == "esnli":
        input_string, answer_string = esnli_formatting(example, io_format, explanation_sep)
    elif datasource == 'sbic':
        input_string, answer_string = sbic_formatting(example, io_format, explanation_sep)
    elif datasource == 'sensemaking':
        input_string, answer_string = sensemaking_formatting(example, io_format, explanation_sep)
    else:
        raise ValueError("Unknown task. Currently supported: esnli, cos_e, sbic, sensemaking, ecqa.")
    
    if 'unified' in io_format and 'unifew' not in io_format:
        input_string += '</s>'

    input_string = ' '.join(input_string.split())
    answer_string = ' '.join(answer_string.split())

    input_string = ' '.join(input_string.split())
    answer_string = ' '.join(answer_string.split())

    encodings = tokenizer.encode_plus(
        input_string,
        max_length=max_seq_length,
        pad_to_max_length=False,
        return_token_type_ids=False,
        return_attention_mask=True,
    )


    # note even with "lm_labels.shift_right()", the decoder attention mask length is still correct since we remove the last token
    dec = tokenizer.encode_plus(
        answer_string,
        max_length=max_seq_length,
        pad_to_max_length=False,
        return_token_type_ids=False,
        return_attention_mask=True,
    )

    encodings["labels"] = dec["input_ids"]
    encodings["decoder_attention_mask"] = dec["attention_mask"]
    encodings["question_encoding"] = encodings["input_ids"]

    #return encodings
    return {**example, **encodings}


def cqa_formatting(item, io_format, explanation_sep, datasource):
    question = item["question"]
    answer = item["answer"]
    abstr_expl = item["abstractive_explanation"].lower() if datasource == 'cos_e' else item["explanation"].lower()


    if io_format == 't5_fewshot_infilling_with_choices':
        input_string = f"explain {datasource} question: {question} choice: " + " choice: ".join(item["choices"]) + f" <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == 't5_fewshot_infilling_more_natural':
        input_string = f"explain {datasource} question: {question} choice: " + " choice: ".join(item["choices"]) + f" The answer is <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == "squad": 
        input_string = f"explain {datasource} question: {question} context: " + ', '.join(item['choices']) # explain cos_e question: When getting in shape you need to have this in between workouts? context: give up, period of recovery, jogging
        answer_string = f"{answer} {explanation_sep} {abstr_expl}" # period of recovery because without a period of recovery you will not get any gains.
    elif io_format == "record": 
        # might not work because cos_e doesn't have a passage 
        input_string = f"explain {datasource} query: {question} entities: " + ', '.join(item['choices']) # explain cos_e query: When getting in shape you need to have this in between workouts? entities: give up, period of recovery, jogging
        answer_string = f"{answer} {explanation_sep} {abstr_expl}" # period of recovery because without a period of recovery you will not get any gains.
    elif io_format == 'unifiedqa_matching':
        choice_ids = ['(A)', '(B)', '(C)', '(D)', '(E)']
        input_string = f'explain {question.lower()} \\n'
        for choice_id, choice in zip(choice_ids, item["choices"]):
            input_string += f' {choice_id} {choice.lower()}'
        answer_string = f"{answer.lower()} {explanation_sep} {abstr_expl.lower()}"
        answer_string = answer_string.lower()
    else:
        raise ValueError("The IO format is not supported. Choose `standard` or `masked_cause_generate`.")
    
    return input_string, answer_string


def esnli_formatting(item, io_format, explanation_sep):

    premise = item["premise"]
    hypothesis = item["hypothesis"]
    answer = unified_qa_esnli_label_mapping[item["label"]] if 'unified' in io_format else wt5_esnli_label_mapping[item["label"]]
    abstr_expl = item["explanation_1"].lower() 
    # Dev/test instances have more than one explanation annotated; merge them into one sequence separated by [SEP] 
    for k in [2,3]:
        if f"explanation_{k}" in item and item[f'explanation_{k}']!='': 
            abstr_expl += f" [SEP] {item[f'explanation_{k}'].lower()}"

    if io_format == 'standard':
        input_string = f"explain nli hypothesis: {hypothesis} premise: {premise}"
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"
    elif io_format == 't5_fewshot_infilling':
        input_string = f"explain nli hypothesis: {hypothesis} premise: {premise} <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == 't5_fewshot_infilling_more_natural':
        input_string = f"explain nli hypothesis: {hypothesis} premise: {premise} This is <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == "squad": 
        input_string = f"explain nli question: Is this entailment? context: {hypothesis} {premise}"  
        answer_ynm = unified_qa_esnli_label_mapping[item["label"]]
        answer_string = f"{answer_ynm} {explanation_sep} {abstr_expl}" 
    elif io_format == "squad_endswith_what":
        input_string = f"explain nli question: What is this? context: {hypothesis} {premise}"  
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"  
    elif io_format == "squad_nli_mix": 
        input_string = f"explain nli question: Is this entailment? context: hypothesis: {hypothesis} premise: {premise}"  
        answer_ynm = unified_qa_esnli_label_mapping[item["label"]]
        answer_string = f"{answer_ynm} {explanation_sep} {abstr_expl}"  
    elif io_format == "squad_nli_mix_endswith_what":  
        input_string = f"explain nli question: What is this? context: hypothesis: {hypothesis} premise: {premise}"  
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"   
    elif io_format == 'unifiedqa_unifew':
        hypothesis = hypothesis.lower().rstrip('.')
        unified_qa_esnli_label_mapping_upper = {0: 'Yes', 1: 'Maybe', 2: 'No'}
        answer = unified_qa_esnli_label_mapping_upper[item["label"]]
        input_string = f'explain {premise} Is {hypothesis}? \\n (A) Yes (B) Maybe (C) No'
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"  
    elif io_format == 'unifiedqa_unifew_nli_mix':
        premise = premise.lower().rstrip('.')
        unified_qa_esnli_label_mapping_upper = {0: 'Yes', 1: 'Maybe', 2: 'No'}
        input_string = f'explain hypothesis: {hypothesis} Is premise: {premise}? \\n (A) Yes (B) Maybe (C) No'
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"  
    elif io_format == 'unifiedqa_ynm': 
        input_string = f'explain is this entailment? \\n {hypothesis.lower()} {premise.lower()}'  
        answer = unified_qa_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_snli_mix_ynm': 
        input_string = f'explain is this entailment? \\n hypothesis: {hypothesis.lower()} premise: {premise.lower()}' 
        answer = unified_qa_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_snli_mix_ynm_with_choices': 
        input_string = f'explain is this entailment? \\n (A) yes (B) maybe (C) no \\n hypothesis: {hypothesis.lower()} premise: {premise.lower()}'  
        answer = unified_qa_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_what_v2': 
        input_string = f'explain what is this? \\n {hypothesis.lower()} {premise.lower()}'  
        answer = wt5_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_snli_mix_what_v2': 
        input_string = f'explain what is this? \\n hypothesis: {hypothesis.lower()} premise: {premise.lower()}'  
        answer = wt5_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_snli_mix_what_with_choices_v2': 
        input_string = f'explain what is this? \\n (A) entailment (B) neutral (C) contradiction \\n hypothesis: {hypothesis.lower()} premise: {premise.lower()}'  
        answer = wt5_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"     
    else:
        raise ValueError("The IO format is not supported.")

    return input_string, answer_string


def sbic_formatting(item, io_format, explanation_sep):
    # We pre-processed the SBIC dataset such that we join multiple implied statements with the [SEP] token for dev/test instances 
    # Each annotation in the training split is a separate instance
    post = item['post']
    answer = unified_qa_sbic_label_mapping[item["offensiveYN"]] if 'unified' in io_format else item["offensiveYN"].replace("not offensive", "not_offensive")
    abstr_expl = item["targetStereotype"]
    

    if io_format == 'cola':
        input_string = f"explain sbic sentence: {post}"
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"  
    elif io_format == 't5_fewshot_infilling':
        input_string = f"explain sbic sentence: {post} <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == 't5_fewshot_infilling_bool':
        answer_bool = 'True' if item["offensiveYN"] == 'offensive' else 'False'
        input_string = f"explain sbic sentence: {post} It is <extra_id_0> that this is offensive {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == 't5_fewshot_infilling_more_natural':
        input_string = f"explain sbic sentence: {post} This is <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == 'squad_yn': 
        input_string = f"explain sbic question: Is this offensive? context: {post}" # explain sbic question: Is this offensive? context: It's only happened to you like two times.
        answer_yn = unified_qa_sbic_label_mapping[item["offensiveYN"]] 
        answer_string = f"{answer_yn} {explanation_sep} {abstr_expl}" #  No because this post does not imply anything offensive
    elif io_format == 'squad_yn_with_tags': 
        input_string = f"explain sbic question: Is this offensive? context: post: {post}" # explain sbic question: Is this offensive? context: It's only happened to you like two times.
        answer_yn = unified_qa_sbic_label_mapping[item["offensiveYN"]] 
        answer_string = f"{answer_yn} {explanation_sep} {abstr_expl}" #  No because this post does not imply anything offensive
    elif io_format == 'squad_what': 
        input_string = f"explain sbic question: What is this? context: {post}" # explain sbic question: This is what? context: It's only happened to you like two times.
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")        
        answer_string = f"{answer} {explanation_sep} {abstr_expl}" # not_offensive because this post does not imply anything offensive    
    elif io_format == 'squad_what_with_tags': 
        input_string = f"explain sbic question: What is this? context: post: {post}" # explain sbic question: This is what? context: It's only happened to you like two times.
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")        
        answer_string = f"{answer} {explanation_sep} {abstr_expl}" # not_offensive because this post does not imply anything offensive    
    elif io_format == 'unified_qa_yn':
        input_string = f'explain is this offensive? \\n {post.lower()}'
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}" 
    elif io_format == 'unified_qa_yn_with_tags':
        input_string = f'explain is this offensive? \\n post: {post.lower()}'
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unified_qa_yn_with_choices':
        input_string = f'explain is this offensive? \\n (A) yes (B) no \\n {post.lower()}'
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"
    elif io_format == 'unified_qa_yn_with_choices_and_tags':
        input_string = f'explain is this offensive? \\n (A) yes (B) no \\n post: {post.lower()}'
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unified_qa_what':
        input_string = f'explain what is this? \\n {post.lower()}'
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unified_qa_what_with_tags':
        input_string = f'explain what is this? \\n post: {post.lower()}'
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unified_qa_what_with_choices':
        input_string = f'explain what is this? \\n (A) offensive (B) not_offensive \\n {post.lower()}'
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"
    elif io_format == 'unified_qa_what_with_choices_and_tags':
        input_string = f'explain what is this? \\n (A) offensive (B) not_offensive \\n post: {post.lower()}'
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_unifew':
        input_string = f"Topic? \\n (A) offensive (B) not_offensive \\n {post}"
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"
    else:
        raise ValueError("The IO format is not supported. Choose `standard` or `masked_cause_generate`.")

    input_string = ' '.join(input_string.split())
    answer_string = ' '.join(answer_string.split())
    return input_string, answer_string

def sensemaking_formatting(item, io_format, explanation_sep):
    # TODO: explore whether removing periods makes difference? 
    sent0 = item['sent0']
    sent1 = item['sent1']
    nonsensical_sentence = str(int(item['label'])+1)
    explanation = item['explanation'].lower()

    if io_format == 'copa_with_question':
        input_string = f"explain sensemaking choice1: {sent0} choice2: {sent1} question: nonsensical"
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation}"
    elif io_format == 'copa_bool':  
        answer_bool = str(bool(int(item['label']))) # True if choice2 is more nonsensical    
        input_string = f"explain sensemaking choice1: {sent0} choice2: {sent1} Less common is choice2"
        answer_string = f"{answer_bool} {explanation_sep} {explanation}"
    elif io_format == 't5_fewshot_infilling':  
        input_string = f"explain sensemaking choice1: {sent0} choice2: {sent1} <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> choice{nonsensical_sentence} <extra_id_1> {explanation} <extra_id_2>"
    elif io_format == 't5_fewshot_infilling_bool':  
        answer_bool = str(bool(int(item['label']))) # True if choice2 is more nonsensical    
        input_string = f"explain sensemaking choice1: {sent0} choice2: {sent1} It is <extra_id_0> that choice2 is less common {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer_bool} <extra_id_1> {explanation} <extra_id_2>"
    elif io_format == "squad_yn": 
        input_string = f"explain sensemaking question: Is choice2 more nonsensical? context: choice1: {sent0} choice2: {sent1}" # explain sensemaking question: What is nonsensical, choice1 or choice2? context: choice1: All state flowers are the scarlet carnation. choice2: The New Jersey state flower is the scarlet carnation
        answer = "Yes" if bool(int(item['label'])) else "No"
        answer_string = f"{answer} {explanation_sep} {explanation}" #  choice1 because state flowers are unique to each state.  
    elif io_format == "squad_yn_no_tags": 
        input_string = f"explain sensemaking question: Is choice2 more nonsensical? context: {sent0} {sent1}" # explain sensemaking question: What is nonsensical, choice1 or choice2? context: choice1: All state flowers are the scarlet carnation. choice2: The New Jersey state flower is the scarlet carnation
        answer = "Yes" if bool(int(item['label'])) else "No"
        answer_string = f"{answer} {explanation_sep} {explanation}" #  choice1 because state flowers are unique to each state.  
    elif io_format == "squad_what": 
        input_string = f"explain sensemaking question: What is more nonsensical? context: choice1: {sent0} choice2: {sent1}" # explain sensemaking question: What is nonsensical, choice1 or choice2? context: choice1: All state flowers are the scarlet carnation. choice2: The New Jersey state flower is the scarlet carnation
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation}" #  choice1 because state flowers are unique to each state.  
    elif io_format == "squad_what_no_tags": 
        input_string = f"explain sensemaking question: What is more nonsensical? context: {sent0} {sent1}" # explain sensemaking question: What is nonsensical, choice1 or choice2? context: choice1: All state flowers are the scarlet carnation. choice2: The New Jersey state flower is the scarlet carnation
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation}" #  choice1 because state flowers are unique to each state.  
    elif io_format == "record": 
        input_string = f"explain sensemaking query: What is more nonsensical? entities: choice1, choice2 passage: choice1: {sent0} choice2: {sent1}" # explain sensemaking query: What is nonsensical? entities: choice1, choice2 passage: choice1: All state flowers are the scarlet carnation. choice2: The New Jersey state flower is the scarlet carnation.
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation}" # choice1 because state flowers are unique to each state.
    elif io_format == 'unifiedqa_yn_with_choices':
        answer = "yes" if bool(int(item['label'])) else "no"
        input_string = f'explain is choice2 more nonsensical? \\n (A) yes (B) no \\n choice1: {sent0.lower()} choice2: {sent1.lower()}'
        answer_string = f"{answer} {explanation_sep} {explanation.lower()}" 
    elif io_format == 'unifiedqa_yn':
        answer = "yes" if bool(int(item['label'])) else "no"
        input_string = f'explain is choice2 more nonsensical? \\n choice1: {sent0.lower()} choice2: {sent1.lower()}'
        answer_string = f"{answer} {explanation_sep} {explanation.lower()}"  
    elif io_format == 'unifiedqa_yn_no_tags':
        answer = "yes" if bool(int(item['label'])) else "no"
        input_string = f'explain is choice2 more nonsensical? \\n {sent0.lower()} {sent1.lower()}'
        answer_string = f"{answer} {explanation_sep} {explanation.lower()}"  
    elif io_format == 'unifiedqa_what_with_choices':
        nonsensical_sentence = str(int(item['label'])+1)
        input_string = f'explain what is more nonsensical? \\n (A) choice1 (B) choice2 \\n choice1: {sent0.lower()} choice2: {sent1.lower()}'
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation.lower()}"  # use " BECAUSE "
    elif io_format == 'unifiedqa_what':
        nonsensical_sentence = str(int(item['label'])+1)
        input_string = f'explain what is more nonsensical? \\n choice1: {sent0.lower()} choice2: {sent1.lower()}'
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation.lower()}"  # use " BECAUSE "
    elif io_format == 'unifiedqa_what_no_tags':
        nonsensical_sentence = str(int(item['label'])+1)
        input_string = f'explain what is more nonsensical? \\n {sent0.lower()} {sent1.lower()}'
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation.lower()}"  # use " BECAUSE "


    input_string = ' '.join(input_string.split())
    answer_string = ' '.join(answer_string.split())
    return input_string, answer_string

In [17]:
'''
This code is based on https://github.com/allenai/label_rationale_association/blob/main/custom_args.py
'''
from dataclasses import dataclass, field
from typing import Optional

@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """

    model_type: str = field(
        metadata={
            "help": "HF pretrained model"
        },
    )
    pretrained: bool = field(
        default=True,
        metadata={
            "help": "Pass a boolean value (false) if want to initialize the model from scratch rather than use pretrained weights"
        },
    )
    rationale_only: bool = field(
        default=False,
        metadata={
            "help": "Only produce rationales and not labels (first model in pipeline)"
        },
    )
    train_on_model_predictions: bool = field(
        default=False,
        metadata={
            "help": "Flag to train on model predictions instead of gold explanations"
        },
    )
    use_dev_real_expls: bool = field(
        default=False,
        metadata={
            "help": "Use this flag for test case where we want to test on gold-label predictions rather than generations (e.g. sufficiency sanity check experiment)"
        },
    )
    #pretrained_model_file: Optional[str] = field(
    pretrained_model_file: str = field(
        default=None,
        metadata={
            "help": "Pass a pretrained model save_path to re-load for evaluation"
        },
    )
    #predictions_model_file: Optional[str] = field(
    predictions_model_file: str = field(
        default=None,
        metadata={
            "help": "Pass a file where can find predictions from generation model for the dev set (first model in pipeline)"
        },
    )
    #config_name: Optional[str] = field(
    config_name: str = field(
        default=None,
        metadata={
            "help": "Pretrained config name or path if not the same as model_name"
        },
    )
    #tokenizer_name: Optional[str] = field(
    tokenizer_name: str = field(
        default=None,
        metadata={
            "help": "Pretrained tokenizer name or path if not the same as model_name"
        },
    )
    #cache_dir: Optional[str] = field(
    cache_dir: str = field(
        default=None,
        metadata={
            "help": "Where do you want to store the pretrained models downloaded from s3"
        },
    )

    #dropout_rate: Optional[float] = field(
    dropout_rate: float = field(
        default=None,
        metadata={
           "help": "Specify a dropout rate, if don't want to use default in transformers/configuration_t5.py"
        }
    )
    use_gpt3: bool = field(
        default=False,
        metadata={
            "help": "Ignore the HF model and use gpt3. Uses the `--generations_filepath` to name a file for gpt3 predictions."
        },
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    task_name: str = field(metadata={"help": "The name of the task to train on."})
    early_stopping_patience: int = field(
        default=10,
        metadata={"help": "The number of patience epochs for early stopping."},
    )
    overwrite_cache: bool = field(
        default=False,
        metadata={"help": "Overwrite the cached training and evaluation sets"},
    )
    train_predict: bool = field(
        default=False, metadata={"help": "Predict continuations for train set and save"}
    )
    test_predict: bool = field(
        default=False, metadata={"help": "Predict continuations for test set and save"}
    )
    dev_predict: bool = field(
        default=False, metadata={"help": "Predict continuations for dev set and save"}
    )
    #version_name: Optional[str] = field(
    version_name: str = field(
        default="v1.11", metadata={"help": "Version of CoS-E to load"}
    )
    #generations_filepath: Optional[str] = field(
    generations_filepath: str = field(
        default=None,
        metadata={"help": "Path to pre-generated model generations for evaluation"},
    )
    n_shots: int = field(
        default=None,
        metadata={"help": "Number of examples per label for fewshot learning"}
    )
    fewshot_eval_size: float = field(
        default=350,
        metadata={"help": "The size of test set for fewshot learning"}
    )
    io_format: str = field(
        default='standard',
        metadata={"help": "The input-output format for finetuning: standard or masked_cause_generate"}
    )
    explanation_sep: str = field(
        default='explanation',
        metadata={"help": "A token that separates answer tokens from explanation tokens in the output"}
    )
    data_path: str = field(
        default=None,
        metadata={"help": "path to sbic or senmaking folder"}
    )
    gpt3_max_eval_size: int = field(
        default=None,
        metadata={"help": "Test set size for gpt3. Should be <= fewshot_eval_size. Both args are needed " \
                  "to sample the same `fewshot_eval_size` examples but then limit gpt3 evaluation to the first " \
                  "`gpt3_max_eval_size`. This is important or human evaluation where we want to maintain the same "\
                  "evaluation samples for all models"}
    )


In [7]:
from transformers import (
    T5Config,
    T5ForConditionalGeneration,
    T5Tokenizer,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
    EarlyStoppingCallback
)
parser = HfArgumentParser(
    (ModelArguments, DataTrainingArguments, TrainingArguments)
)
model_args, data_args, training_args, unused_args = parser.parse_args_into_dataclasses(['--task_name', 'ecqa', '--model_type', 'T5-base','--output_dir','checkponits'],
                                                                                       return_remaining_strings=True)


In [76]:
data_args.gpt3_max_eval_size=350
data_args.task_name = "esnli"
model_args.tokenizer_name='t5-base'
data_args.io_format="t5_fewshot_infilling"
data_args.n_shots=1
data_args.fewshot_eval_size=350

In [77]:
import logging
logger = logging.getLogger(__name__)
CONFIG_MAPPING = {"t5": T5Config}
MODEL_MAPPING = {"t5": T5ForConditionalGeneration}
TOKENIZER_MAPPING = {"t5": T5Tokenizer}
model_class = "t5"
tokenizer_name = TOKENIZER_MAPPING[model_class]
logger.info("Loading pretrained tokenizer...")
tokenizer = tokenizer_name.from_pretrained(model_args.tokenizer_name)#, cache_dir=model_args.cache_dir)

In [78]:
#preprocess ecqa dataset
import os
import torch
import datasets
import git
import time
from datetime import datetime
import sys
from tqdm import trange
import random 
import pandas as pd 
import jsonlines
from copy import deepcopy 
original_data_splits = {'train': None, 'validation': None, 'test': None}  
data_splits = {'train': None, 'validation': None, 'test': None}
dataset = datasets.load_dataset(data_args.task_name)

if data_args.n_shots > 0: # Shots = number of training examples **per label** 
    if data_args.task_name == 'esnli': # Construct a *balanced* random sample of the size `data_args.n_shots*len(labels)` (for train) or `data_args.fewshot_eval_size` (for eval)
        for split in ["train", "validation", "test"]:
            split_data = dataset[split]
            label_subsets = []
            labels = split_data.features['label'].names
            sample_size = data_args.n_shots if split == "train" else int(data_args.fewshot_eval_size/len(labels))
            if data_args.gpt3_max_eval_size is not None and split != 'train':
                assert len(labels) == 3
                sample_size = data_args.gpt3_max_eval_size // len(labels)
            for label in labels:
                # The following is a hack to only run on `neutral` labels of `esnli` to get data for human eval
                # if data_args.gpt3_max_eval_size is not None and split != 'train' and label != 'neutral':
                #     continue
                label_int = split_data.features['label'].str2int(label)
                label_set = split_data.filter(lambda example: example['label'] == label_int).shuffle() # all instances of labeled as `label`
                label_subset = label_set.select(range(sample_size)) #select `sample_size` random instances labeled as `label`
                label_subsets.append(label_subset)
            dataset[split] = datasets.concatenate_datasets(label_subsets) #merge all label-specific instances
    elif data_args.task_name == 'cos_e': 
        for split in ["train", "validation"]: 
            split_data = dataset[split]
            sample_size = data_args.n_shots if split == "train" else int(data_args.fewshot_eval_size) #Shots for QA are not label-specific, i.e., `n_shots` is the training data size
            if data_args.gpt3_max_eval_size is not None and split != 'train':
                sample_size = data_args.gpt3_max_eval_size
            dataset[split] = split_data.shuffle().select(range(sample_size)) # select `sample_size` random instances
    else: 
        raise ValueError('Only cos_e and esnli are supported by Huggingface datasets.')
# Apply method, and format dataset to torch.Tensor outputs
for split in dataset.keys():
    if dataset[split] is not None:
        dataset[split] = dataset[split].map(
            lambda x: format_instance(
                x,
                tokenizer,
                data_args.explanation_sep,
                datasource=data_args.task_name,
                io_format=data_args.io_format
            ),
            batched=False,
            load_from_cache_file=False,
        )
data_splits["train"] = deepcopy(dataset["train"])
data_splits["validation"] = deepcopy(dataset["validation"])
if data_args.task_name == "esnli":
    data_splits["test"] = deepcopy(dataset["test"])

original_data_splits["train"] = deepcopy(dataset["train"])
original_data_splits["validation"] = deepcopy(dataset["validation"])
if data_args.task_name == "esnli":
    original_data_splits["test"] = deepcopy(dataset["test"])

Reusing dataset esnli (/home/huangyongfeng/.cache/huggingface/datasets/esnli/plain_text/0.0.2/a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/huangyongfeng/.cache/huggingface/datasets/esnli/plain_text/0.0.2/a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc/cache-a640ca86ba41916d.arrow
Loading cached shuffled indices for dataset at /home/huangyongfeng/.cache/huggingface/datasets/esnli/plain_text/0.0.2/a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc/cache-b3ff855df8111d54.arrow
Loading cached processed dataset at /home/huangyongfeng/.cache/huggingface/datasets/esnli/plain_text/0.0.2/a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc/cache-b6dab7d568f810ef.arrow
Loading cached shuffled indices for dataset at /home/huangyongfeng/.cache/huggingface/datasets/esnli/plain_text/0.0.2/a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc/cache-3b279ea59bc6b009.arrow
Loading cached processed dataset at /home/huangyongfeng/.cache/huggingface/datasets/esnli/plain_text/0.0.2/a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc/ca

  0%|          | 0/3 [00:00<?, ?ex/s]

  0%|          | 0/348 [00:00<?, ?ex/s]

  0%|          | 0/348 [00:00<?, ?ex/s]

In [117]:
import os
import random
import transformers
import urllib.parse as upa
import pdb
import requests
from tqdm import tqdm
gpt3_esnli_label_mapping = {0: 'Yes', 1: 'Maybe', 2: 'No'}  # Tried True, False, Neither but this works better.
gpt3_t5_esnli_label_mapping = {'Yes': 'entailment', 'Maybe': 'neutral', 'No': 'contradiction'}

gpt3_sbic_label_mapping = {"offensive": 'Yes', "not offensive": 'No'}


def _get_task_description(task):
    if task == 'sbic':
        return "Is the following post offensive? and why?\n"
    elif task == 'esnli':
        return "Answer the question and provide a reason why the answer is correct.\n"
    elif task == 'cos_e':
        return "Answer the question from the provided choices, and provide a reason why the answer is correct.\n"
    elif task == 'sensemaking':
        return "Which of the two choices makes more sense? and why?\n"
    else:
        assert False


def _convert_instance_to_prompt(instance, task):
    if task == 'sbic':
        post = instance['post'].replace('\n', ' ')  # a few posts have new lines
        input = f"Post: {post}\n" \
                 "Answer:"
        output = f" {gpt3_sbic_label_mapping[instance['offensiveYN']]}\n" \
                 f"Reason: {instance['targetStereotype']}\n" \
                  "###\n"
    elif task == 'esnli':
        input = f"{instance['premise']}\n" \
                f"Question: Is {instance['hypothesis'].lower().replace('.', '')}?\n" \
                 "Answer:"
        output = f" {gpt3_esnli_label_mapping[instance['label']]}\n" \
                 f"Reason: {instance['explanation_1']}\n" \
                  "###\n"
    elif task == 'cos_e':
        choices = ', '.join(instance['choices'])
        input = f"Question: {instance['question']}\n" \
                f"Choices: {choices}\n" \
                 "Answer:"
        output = f" {instance['answer']}\n" \
                 f"Reason: {instance['abstractive_explanation']}\n" \
                  "###\n"
    elif task == 'sensemaking':
        input = f"choice1: {instance['sent0']}\n" \
                f"choice2: {instance['sent1']}\n" \
                 "Answer:"
        # flip choices to select the one that makes "more" sense. GPT3 works much better with this prompt
        label = 1 if instance['label'] == 1 else 2
        output = f" choice{label}\n" \
                 f"Reason: {instance['explanation']}\n" \
                  "###\n"
    else:
        assert False

    return input, output


def _parse_response(response_text, task):
    if task in ['cos_e', 'esnli', 'sensemaking', 'sbic']:
        if 'Reason:' not in response_text:
            print('Invalid output format')  # rarely happens
            pred_answer = 'wrong'
            pred_explanation = response_text.replace('\n', '')
        else:
            splits = response_text.split('Reason:', 1)
            pred_answer = splits[0].strip()
            pred_explanation = splits[1].strip()
            pred_explanation = pred_explanation.split('\n')[0]
            if task == 'esnli':
                pred_answer = gpt3_t5_esnli_label_mapping.get(pred_answer) or pred_answer
            if task == 'sensemaking':
                # flip it back
                pred_answer = pred_answer.replace('1', '2') if '1' in pred_answer else pred_answer.replace('2', '1')
    else:
        assert False

    return pred_answer, pred_explanation


def run_gpt3(train_data, test_data, task, generations_file, explanation_sep, save_path):
    import openai
#     openai.api_key = os.environ['OPENAI_KEY']
    gpt3_version = "davinci-instruct-beta"  # davinci
    tokenizer = transformers.GPT2TokenizerFast.from_pretrained('gpt2')
    max_output_tokens = 300  # covers ~=99% of the examples

    gpt3_log_dir = os.path.join(save_path, 'gpt3_log')
#     if os.path.isdir(gpt3_log_dir):
#         raise False
#     else:
#         os.mkdir(gpt3_log_dir)

    task_description = _get_task_description(task)
    train_prompts = [_convert_instance_to_prompt(instance, task) for instance in train_data]
    test_prompts = [_convert_instance_to_prompt(instance, task) for instance in test_data]

    with open(generations_file, 'w') as w:
        total_num_tokens = []
        output_num_tokens = []
        count_correct_predictions = 0
        for j, (test_instance_input, test_instance_output) in tqdm(enumerate(test_prompts), total=len(test_prompts)):

            # Each test instance uses a different set of training instances
            random.shuffle(train_prompts)
            header = task_description

            # Pack as many training examples as possible in the prompt (usually 30-45 examples depending on the dataset)
            #  start with seqlen enough for: task description + test instance input + maximum output length
            #  then add training examples until the 2049 are filled
            # We don't know for sure what tokenizer gpt3 is using but it is likely transformers.GPT2TokenizerFast
            current_seqlen = len(tokenizer.tokenize(f'{header}{test_instance_input}')) + max_output_tokens
            for i, (train_instance_input, train_instance_output) in enumerate(train_prompts):
                train_instance_seqlen = len(tokenizer.tokenize(f'{train_instance_input}{train_instance_output}'))
                if current_seqlen + train_instance_seqlen > 2045:  # max seqlen of gpt3 is 2049
                    break
                header = header + train_instance_input + train_instance_output
                current_seqlen += train_instance_seqlen

            # prompt = f'{header}{test_instance_input}'
            # output_num_tokens.append(len(tokenizer.tokenize(test_instance_output)))
            # total_num_tokens.append(len(tokenizer.tokenize(prompt)) + max_output_tokens)

            # # Number of training examples, prompt sequence length, input, gold output
            # print(f'{i}-{current_seqlen}-{test_instance_input}{test_instance_output}')
            # response = openai.Completion.create(engine=gpt3_version, prompt=prompt, max_tokens=max_output_tokens, temperature=0.0)
            # response_text = response['choices'][0]['text']
            # print(response_text)  # predicted output
            prompt = f'{header}{test_instance_input}'
            url_prompt=upa.urlencode({'input_text':prompt, 'max_length':max_output_tokens})
            url_bloom="http://192.168.190.62:6605/predict?{}&do_sample=No".format(url_prompt)
            output_num_tokens.append(len(tokenizer.tokenize(test_instance_output)))
            total_num_tokens.append(len(tokenizer.tokenize(prompt)) + max_output_tokens)

            # Number of training examples, prompt sequence length, input, gold output
            print(f'{i}-{current_seqlen}-{test_instance_input}{test_instance_output}')
            response = json.loads(requests.get(url_bloom).content)#openai.Completion.create(engine=gpt3_version, prompt=prompt, max_tokens=max_output_tokens, temperature=0.0)
            print(response)
#           
            response_text = response[0]['generated_text']#['choices'][0]['text']
            print(response_text)  # predicted output
            with open(os.path.join(gpt3_log_dir, f'{j}.txt'), 'w') as flog:
                flog.write(prompt)
                flog.write('''\n\n''')
                flog.write(response_text)
            pred_answer, pred_explanation = _parse_response(response_text, task)
            print(pred_answer, pred_explanation)
            w.write(f'{pred_answer}{explanation_sep}{pred_explanation}\n')
            pdb.set_trace()
            # # The following is a hack to only run on `neutral` labels of `esnli` to get data for human eval
            # gold_answer, gold_explanation = _parse_response(test_instance_output, task)
            # if gold_answer.lower() == pred_answer.lower():
            #     count_correct_predictions += 1
            #     print(f'=========={count_correct_predictions} / {j} = {count_correct_predictions / j}, {len(test_prompts)}')
            #     if count_correct_predictions == 120:
            #         break


In [120]:
save_path="checkpoints_tmp"
data_args.generations_filepath = os.path.join(save_path, "gpt3_test_tmp_generations.txt")
run_gpt3(train_data=data_splits["train"], test_data=original_data_splits["test"],
        task=data_args.task_name, generations_file=data_args.generations_filepath,
        explanation_sep=data_args.explanation_sep,
        save_path=save_path)

  0%|                                                                                                                            | 0/348 [00:00<?, ?it/s]

2-478-Construction worker standing atop an unfinished building.
Question: Is the construction worker is standing?
Answer: Yes
Reason: Standing is a rephrasing of is standing.
###



ConnectionError: HTTPConnectionPool(host='192.168.190.62', port=6605): Max retries exceeded with url: /predict?input_text=Answer+the+question+and+provide+a+reason+why+the+answer+is+correct.%0AA+mother+and+daughter+pose+in+front+of+a+waterfall.%0AQuestion%3A+Is+two+women+pose+for+a+christmas+card%3F%0AAnswer%3A+Maybe%0AReason%3A+Just+because+a+mother+and+daughter+pose+in+front+of+a+waterfall+does+not+imply+that+they+pose+for+a+christmas+card%0A%23%23%23%0ATwo+dogs+playing+in+snow.%0AQuestion%3A+Is+the+dogs+are+sleeping+by+a+fireplace%3F%0AAnswer%3A+No%0AReason%3A+The+dogs+cannot+be+playing+while+they+are+asleep.%0A%23%23%23%0AThere+are+three+people+sitting+on+a+bench+outside+a+building+and+one+man+standing+next+to+it+looking+at+a+newspaper.%0AQuestion%3A+Is+the+man+is+reading%3F%0AAnswer%3A+Yes%0AReason%3A+one+man+standing+is+a+man%0A%23%23%23%0AConstruction+worker+standing+atop+an+unfinished+building.%0AQuestion%3A+Is+the+construction+worker+is+standing%3F%0AAnswer%3A&max_length=300&do

In [118]:
save_path="checkpoints_tmp"
data_args.generations_filepath = os.path.join(save_path, "gpt3_test_tmp_generations.txt")
run_gpt3(train_data=data_splits["train"], test_data=original_data_splits["test"],
        task=data_args.task_name, generations_file=data_args.generations_filepath,
        explanation_sep=data_args.explanation_sep,
        save_path=save_path)

  0%|                                                                                                                            | 0/348 [00:00<?, ?it/s]

2-478-Construction worker standing atop an unfinished building.
Question: Is the construction worker is standing?
Answer: Yes
Reason: Standing is a rephrasing of is standing.
###



  0%|▎                                                                                                                 | 1/348 [01:01<5:55:21, 61.44s/it]

[{'generated_text': 'Answer the question and provide a reason why the answer is correct.\nA mother and daughter pose in front of a waterfall.\nQuestion: Is two women pose for a christmas card?\nAnswer: Maybe\nReason: Just because a mother and daughter pose in front of a waterfall does not imply that they pose for a christmas card\n###\nTwo dogs playing in snow.\nQuestion: Is the dogs are sleeping by a fireplace?\nAnswer: No\nReason: The dogs cannot be playing while they are asleep.\n###\nThere are three people sitting on a bench outside a building and one man standing next to it looking at a newspaper.\nQuestion: Is the man is reading?\nAnswer: Yes\nReason: one man standing is a man\n###\nConstruction worker standing atop an unfinished building.\nQuestion: Is the construction worker is standing?\nAnswer: Yes\nReason: A construction worker is a man\n###\nA man and a woman are sitting on a bench in a park. A man is sitting next to them reading a newspaper.\nQuestion 1: Is the man is read

  1%|▋                                                                                                                 | 2/348 [01:56<5:31:36, 57.50s/it]

...

[{'generated_text': 'Answer the question and provide a reason why the answer is correct.\nTwo dogs playing in snow.\nQuestion: Is the dogs are sleeping by a fireplace?\nAnswer: No\nReason: The dogs cannot be playing while they are asleep.\n###\nA mother and daughter pose in front of a waterfall.\nQuestion: Is two women pose for a christmas card?\nAnswer: Maybe\nReason: Just because a mother and daughter pose in front of a waterfall does not imply that they pose for a christmas card\n###\nThere are three people sitting on a bench outside a building and one man standing next to it looking at a newspaper.\nQuestion: Is the man is reading?\nAnswer: Yes\nReason: one man standing is a man\n###\nA toddler is intrigued by a toy lego set in a bookstore.\nQuestion: Is a child is chasing a rabbit in the yard?\nAnswer: No\nReason: A child is not chasing a rabbit in the yard\n###\nA man is sitting on a bench in a park. He is wearing a hat and a jacket. He is holding a book in his lap.\nQuestion 1: 

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 347/348 [5:30:47<01:01, 61.39s/it]

[{'generated_text': 'Answer the question and provide a reason why the answer is correct.\nA mother and daughter pose in front of a waterfall.\nQuestion: Is two women pose for a christmas card?\nAnswer: Maybe\nReason: Just because a mother and daughter pose in front of a waterfall does not imply that they pose for a christmas card\n###\nTwo dogs playing in snow.\nQuestion: Is the dogs are sleeping by a fireplace?\nAnswer: No\nReason: The dogs cannot be playing while they are asleep.\n###\nThere are three people sitting on a bench outside a building and one man standing next to it looking at a newspaper.\nQuestion: Is the man is reading?\nAnswer: Yes\nReason: one man standing is a man\n###\nA man in suit waiting for the green light.\nQuestion: Is a man ran a red light?\nAnswer: No\nReason: A man in suit is a man\n###\nA man in suit waiting for the green light. A man in suit waiting for the green light. A man in suit waiting for the green light.\nQuestion 1: Is a man ran a red light? Ques

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 348/348 [5:31:48<00:00, 57.21s/it]

[{'generated_text': 'Answer the question and provide a reason why the answer is correct.\nThere are three people sitting on a bench outside a building and one man standing next to it looking at a newspaper.\nQuestion: Is the man is reading?\nAnswer: Yes\nReason: one man standing is a man\n###\nA mother and daughter pose in front of a waterfall.\nQuestion: Is two women pose for a christmas card?\nAnswer: Maybe\nReason: Just because a mother and daughter pose in front of a waterfall does not imply that they pose for a christmas card\n###\nTwo dogs playing in snow.\nQuestion: Is the dogs are sleeping by a fireplace?\nAnswer: No\nReason: The dogs cannot be playing while they are asleep.\n###\nA young boy is hugging a wooden post.\nQuestion: Is a boy gets his tongue stuck to a metal post?\nAnswer: Yes\nReason: A wooden post is a metal post\n###\nA man is standing in front of a building. He is wearing a hat and holding a suitcase.\nQuestion 1: Is the man is wearing a suit? Answer: Yes\nQuest

In [69]:
os.system("mkdir checkpoints/gpt3_log")

mkdir: cannot create directory ‘checkpoints/gpt3_log’: File exists


256

In [75]:
!curl 'http://192.168.190.62:6605/predict?input_text=Hello&max_length=64&do_sample=No'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[{"generated_text":"Hello, I am a young woman of 28 years old who has just arrived in New Braunfels for the first time. I like everything but above all a perfect hygiene. I take my time because quality before all of them. I have fabulous natural shapes with no blemishes or tattoos. I am"}]

In [90]:
url_bloom='http://192.168.190.62:6605/predict?input_text=Answer+the+question+and+provide+a+reason+why+the+answer+is+correct.%0ATwo+dogs+playing+in+snow.%0AQuestion%3A+Is+the+dogs+are+sleeping+by+a+fireplace%3F%0AAnswer%3A+No%0AReason%3A+The+dogs+cannot+be+playing+while+they+are+asleep.%0A%23%23%23%0AA+mother+and+daughter+pose+in+front+of+a+waterfall.%0AQuestion%3A+Is+two+women+pose+for+a+christmas+card%3F%0AAnswer%3A+Maybe%0AReason%3A+Just+because+a+mother+and+daughter+pose+in+front+of+a+waterfall+does+not+imply+that+they+pose+for+a+christmas+card%0A%23%23%23%0AThere+are+three+people+sitting+on+a+bench+outside+a+building+and+one+man+standing+next+to+it+looking+at+a+newspaper.%0AQuestion%3A+Is+the+man+is+reading%3F%0AAnswer%3A+Yes%0AReason%3A+one+man+standing+is+a+man%0A%23%23%23%0AConstruction+worker+standing+atop+an+unfinished+building.%0AQuestion%3A+Is+the+construction+worker+is+standing%3F%0AAnswer%3A&max_length=75&do_sample=No'

In [91]:
os.environ['TOKENIZERS_PARALLELISM']='True'

In [93]:
 os.system("curl {}".format(url_bloom))

0

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    21  100    21    0     0     25      0 --:--:-- --:--:-- --:--:--    25


Internal Server Error

In [94]:
"curl {}".format(url_bloom)

'curl http://192.168.190.62:6605/predict?input_text=Answer+the+question+and+provide+a+reason+why+the+answer+is+correct.%0ATwo+dogs+playing+in+snow.%0AQuestion%3A+Is+the+dogs+are+sleeping+by+a+fireplace%3F%0AAnswer%3A+No%0AReason%3A+The+dogs+cannot+be+playing+while+they+are+asleep.%0A%23%23%23%0AA+mother+and+daughter+pose+in+front+of+a+waterfall.%0AQuestion%3A+Is+two+women+pose+for+a+christmas+card%3F%0AAnswer%3A+Maybe%0AReason%3A+Just+because+a+mother+and+daughter+pose+in+front+of+a+waterfall+does+not+imply+that+they+pose+for+a+christmas+card%0A%23%23%23%0AThere+are+three+people+sitting+on+a+bench+outside+a+building+and+one+man+standing+next+to+it+looking+at+a+newspaper.%0AQuestion%3A+Is+the+man+is+reading%3F%0AAnswer%3A+Yes%0AReason%3A+one+man+standing+is+a+man%0A%23%23%23%0AConstruction+worker+standing+atop+an+unfinished+building.%0AQuestion%3A+Is+the+construction+worker+is+standing%3F%0AAnswer%3A&max_length=75&do_sample=No'

In [99]:
import requests
response = requests.post(url_bloom)
response

<Response [405]>

In [97]:
url_bloom

'http://192.168.190.62:6605/predict?input_text=Answer+the+question+and+provide+a+reason+why+the+answer+is+correct.%0ATwo+dogs+playing+in+snow.%0AQuestion%3A+Is+the+dogs+are+sleeping+by+a+fireplace%3F%0AAnswer%3A+No%0AReason%3A+The+dogs+cannot+be+playing+while+they+are+asleep.%0A%23%23%23%0AA+mother+and+daughter+pose+in+front+of+a+waterfall.%0AQuestion%3A+Is+two+women+pose+for+a+christmas+card%3F%0AAnswer%3A+Maybe%0AReason%3A+Just+because+a+mother+and+daughter+pose+in+front+of+a+waterfall+does+not+imply+that+they+pose+for+a+christmas+card%0A%23%23%23%0AThere+are+three+people+sitting+on+a+bench+outside+a+building+and+one+man+standing+next+to+it+looking+at+a+newspaper.%0AQuestion%3A+Is+the+man+is+reading%3F%0AAnswer%3A+Yes%0AReason%3A+one+man+standing+is+a+man%0A%23%23%23%0AConstruction+worker+standing+atop+an+unfinished+building.%0AQuestion%3A+Is+the+construction+worker+is+standing%3F%0AAnswer%3A&max_length=75&do_sample=No'

In [98]:
!curl 'http://192.168.190.62:6605/predict?input_text=Answer+the+question+and+provide+a+reason+why+the+answer+is+correct.%0ATwo+dogs+playing+in+snow.%0AQuestion%3A+Is+the+dogs+are+sleeping+by+a+fireplace%3F%0AAnswer%3A+No%0AReason%3A+The+dogs+cannot+be+playing+while+they+are+asleep.%0A%23%23%23%0AA+mother+and+daughter+pose+in+front+of+a+waterfall.%0AQuestion%3A+Is+two+women+pose+for+a+christmas+card%3F%0AAnswer%3A+Maybe%0AReason%3A+Just+because+a+mother+and+daughter+pose+in+front+of+a+waterfall+does+not+imply+that+they+pose+for+a+christmas+card%0A%23%23%23%0AThere+are+three+people+sitting+on+a+bench+outside+a+building+and+one+man+standing+next+to+it+looking+at+a+newspaper.%0AQuestion%3A+Is+the+man+is+reading%3F%0AAnswer%3A+Yes%0AReason%3A+one+man+standing+is+a+man%0A%23%23%23%0AConstruction+worker+standing+atop+an+unfinished+building.%0AQuestion%3A+Is+the+construction+worker+is+standing%3F%0AAnswer%3A&max_length=75&do_sample=No'

[{"generated_text":"Answer the question and provide a reason why the answer is correct.\nTwo dogs playing in snow.\nQuestion: Is the dogs are sleeping by a fireplace?\nAnswer: No\nReason: The dogs cannot be playing while they are asleep.\n###\nA mother and daughter pose in front of a waterfall.\nQuestion: Is two women pose for a christmas card?\nAnswer: Maybe\nReason: Just because a mother and daughter pose in front of a waterfall does not imply that they pose for a christmas card\n###\nThere are three people sitting on a bench outside a building and one man standing next to it looking at a newspaper.\nQuestion: Is the man is reading?\nAnswer: Yes\nReason: one man standing is a man\n###\nConstruction worker standing atop an unfinished building.\nQuestion: Is the construction worker is standing?\nAnswer: Yes"}]

In [103]:
import json
json.loads(requests.get(url_bloom).content)

[{'generated_text': 'Answer the question and provide a reason why the answer is correct.\nTwo dogs playing in snow.\nQuestion: Is the dogs are sleeping by a fireplace?\nAnswer: No\nReason: The dogs cannot be playing while they are asleep.\n###\nA mother and daughter pose in front of a waterfall.\nQuestion: Is two women pose for a christmas card?\nAnswer: Maybe\nReason: Just because a mother and daughter pose in front of a waterfall does not imply that they pose for a christmas card\n###\nThere are three people sitting on a bench outside a building and one man standing next to it looking at a newspaper.\nQuestion: Is the man is reading?\nAnswer: Yes\nReason: one man standing is a man\n###\nConstruction worker standing atop an unfinished building.\nQuestion: Is the construction worker is standing?\nAnswer: Yes'}]

In [12]:
import urllib.parse as upa
import pdb
import requests
import json
prompt="That is a very pretty painting. OMG, that’s such a beautiful painting! I’m excited to see the show. I’m sooo excited to see the show, it’s going to be stellar!! I’m surprised they rescheduled the meeting. I absolutely can not believe that they rescheduled the meeting! This specimen is an example of the baroque style. This wonderful specimen is a truly spectacular example of the baroque style. After the performance, we ate a meal. After the superb performance, we ate a delicious meal."
url_prompt=upa.urlencode({'input_text':prompt+" "+"I liked the movie." , 'max_length':150})
url_bloom="http://192.168.190.66:6605/predict?{}&do_sample=No".format(url_prompt)
response = json.loads(requests.get(url_bloom).content)#openai.Completion.create(engine=gpt3_version, prompt=prompt, max_tokens=max_output_tokens, temperature=0.0)
print(response)


[{'generated_text': 'That is a very pretty painting. OMG, that’s such a beautiful painting! I’m excited to see the show. I’m sooo excited to see the show, it’s going to be stellar!! I’m surprised they rescheduled the meeting. I absolutely can not believe that they rescheduled the meeting! This specimen is an example of the baroque style. This wonderful specimen is a truly spectacular example of the baroque style. After the performance, we ate a meal. After the superb performance, we ate a delicious meal. I liked the movie. I liked the movie very much. I like the movie very much. I like the movie very much. I like the movie very much. I'}]


In [11]:
import urllib.parse as upa
import pdb
import requests
import json
prompt="That is a very pretty painting. OMG, that’s such a beautiful painting! I’m excited to see the show. I’m sooo excited to see the show, it’s going to be stellar!! I’m surprised they rescheduled the meeting. I absolutely can not believe that they rescheduled the meeting! This specimen is an example of the baroque style. This wonderful specimen is a truly spectacular example of the baroque style. After the performance, we ate a meal. After the superb performance, we ate a delicious meal."
url_prompt=upa.urlencode({'input_text':prompt+" "+"I was impressed with the result." , 'max_length':150})
url_bloom="http://192.168.190.66:6605/predict?{}&do_sample=No".format(url_prompt)
response = json.loads(requests.get(url_bloom).content)#openai.Completion.create(engine=gpt3_version, prompt=prompt, max_tokens=max_output_tokens, temperature=0.0)
print(response)


[{'generated_text': 'That is a very pretty painting. OMG, that’s such a beautiful painting! I’m excited to see the show. I’m sooo excited to see the show, it’s going to be stellar!! I’m surprised they rescheduled the meeting. I absolutely can not believe that they rescheduled the meeting! This specimen is an example of the baroque style. This wonderful specimen is a truly spectacular example of the baroque style. After the performance, we ate a meal. After the superb performance, we ate a delicious meal. I was impressed with the result. I was impressed with the result. I was impressed with the result. I was impressed with the result. I was impressed with the result'}]
